In [1]:
import pandas as pd
from gui import get_values

df = pd.read_csv('predicted_emotions.csv')

ui_arousal, ui_valence = get_values()

# Example UI values for arousal and valence


# Function to calculate Manhattan distance
def calculate_manhattan_distance(x, y, valence, arousal):
    return abs(x - valence) + abs(y - arousal)

# Add a new column to the DataFrame for the Manhattan distance
df['distance'] = df.apply(lambda row: calculate_manhattan_distance(row['Valence'], row['Arousal'], ui_valence, ui_arousal), axis=1)

# Sort the DataFrame based on the 'distance' column and select the top 10 entries
closest_songs = df.sort_values(by='distance').head(10)

# closest matches
closest_song_ids = closest_songs['Song_ID'].tolist()

# Print  list 
print(closest_song_ids)

[16158.0, 10440.0, 12351.0, 33477.0, 3266.0, 31391.0, 29180.0, 10674.0, 3274.0, 890.0]


In [2]:
# 读取CSV的前三行用于生成列名
header_df = pd.read_csv('FMA_Metadata/tracks.csv', nrows=3, header=None)

# 合成列名：合并三行数据，用下划线连接非空非"nan"值
column_names = []
for col in header_df:
    # 过滤出非nan值，转换为字符串
    parts = header_df[col].dropna().apply(lambda x: str(x) if str(x) != 'nan' else '').tolist()
    # 移除空字符串
    parts = [part for part in parts if part]
    # 使用下划线连接
    column_name = '_'.join(parts)
    column_names.append(column_name)

# # 打印合成的列名检查是否正确
# print(column_names)

# 根据需要调整skiprows的值：根据CSV文件的实际内容调整，这里假设数据从第四行开始
df = pd.read_csv('FMA_Metadata/tracks.csv', skiprows=3, header=None, names=column_names)

# 假设第一列是 'track_id'，设置为索引
df.set_index('track_id', inplace=True)

# 歌曲ID列表
song_ids = closest_song_ids

# 过滤出对应歌曲ID的行
filtered_df = df[df.index.isin(song_ids)]

# 提取歌曲名、歌手名等相关信息
# 假设 'track_title' 列代表歌曲名，'artist_name' 列代表歌手名
songs_info = filtered_df[['artist_name', 'track_title']]

# 显示结果
print(songs_info)


                              artist_name  \
track_id                                    
890                           Ilyas Ahmed   
3266                               Ai Aso   
3274                              Arborea   
10440                           Big Blood   
10674                     Nine Inch Nails   
12351                        Koonda Holaa   
16158     Entertainment for the Braindead   
29180                    Charlene Darling   
31391                           Amy Sayer   
33477                          Ran Slavin   

                                                track_title  
track_id                                                     
890                                         Softly Tomorrow  
3266                                                   Land  
3274                                             Forewarned  
10440                                      No Gravity Blues  
10674                                     Lights in the Sky  
12351                    

In [3]:
import os
import pygame
from ipywidgets import Button, Output, HBox, VBox

def play_audio(file_path):
    # 加载音频文件
    pygame.mixer.music.load(file_path)
    
    # 播放音频文件
    pygame.mixer.music.play()

def pause_audio():
    # 暂停音频播放
    pygame.mixer.music.pause()

def unpause_audio():
    # 恢复音频播放
    pygame.mixer.music.unpause()

def find_audio_files(base_path, file_ids):
    audio_files = []
    
    # 遍历 base_path 下的所有子目录
    for root, dirs, files in os.walk(base_path):
        # 检查每个文件是否在 file_ids 中
        for file in files:
            # 假设文件名格式为 '23862.mp3'，提取前面的数字部分
            file_id = os.path.splitext(file)[0]
            try:
                # 将文件名转换为浮点数，并检查是否在列表中
                if float(file_id) in file_ids:
                    file_path = os.path.join(root, file)
                    audio_files.append(file_path)
            except ValueError:
                # 如果转换失败，忽略该文件
                continue
    
    return audio_files

def create_audio_controls(track_id, file_path):
    play_button = Button(description=f"Play: {track_id}")
    pause_button = Button(description="Stop")
    output = Output()
    
    def on_play_button_clicked(b):
        with output:
            print(f"Playing: {track_id}")
            play_audio(file_path)
    
    def on_pause_button_clicked(b):
        if pygame.mixer.music.get_busy():
            if pygame.mixer.music.get_pos() > 0:
                with output:
                    print("Pausing audio")
                pause_audio()
            else:
                with output:
                    print("Resuming audio")
                unpause_audio()
    
    play_button.on_click(on_play_button_clicked)
    pause_button.on_click(on_pause_button_clicked)
    
    return VBox([HBox([play_button, pause_button]), output])


# 初始化 Pygame 混音器
pygame.mixer.init()

# 指定音频文件所在的根目录
base_path = 'FMA_Excerpt'

# 指定要播放的文件ID列表
file_ids = closest_song_ids

# 查找符合条件的音频文件
audio_files = find_audio_files(base_path, file_ids)

# 为每个音频文件创建播放和暂停按钮
audio_controls = [create_audio_controls(os.path.splitext(os.path.basename(file_path))[0], file_path) for file_path in audio_files]

# 显示所有音频控件
VBox(audio_controls)

pygame 2.5.2 (SDL 2.28.3, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
